# Neural Machine Translation by Jointly Learning to Align and Translate

In this third notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473). This model achives our best perplexity yet, ~27 compared to ~34 for the previous model.

## Introduction

Here is the general encoder-decoder model:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

In the previous model, our architecture was set-up in a way to reduce "information compression" by explicitly passing the context vector, $z$, to the decoder at every time-step and by passing both the context vector and embedded input word, $d(y_t)$, along with the hidden state, $s_t$, to the linear layer, $f$, to make a prediction.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Even though we have reduced some of this compression, our context vector still needs to contain all of the information about the source sentence. The model implemented in this notebook avoids this compression by allowing the decoder to look at the entire source sentence (via its hidden states) at each decoding step! How does it do this? It uses *attention*. 

Attention works by first, calculating an attention vector, $a$, that is the length of the source sentence. The attention vector has the property that each element is between 0 and 1, and the entire vector sums to 1. We then calculate a weighted sum of our source sentence hidden states, $H$, to get a weighted source vector, $w$. 

$$w = \sum_{i}a_ih_i$$

We calculate a new weighted source vector every time-step when decoding, using it as input to our decoder RNN as well as the linear layer to make a prediction. We'll explain how to do all of this during the session.


## Import statements

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm_notebook as tqdm

from torchtext.datasets import Multi30k
from torchtext import data
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np
import pandas as pd

import random
import math
import time
import json

Set the random seeds for reproducability.

In [20]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
# downloading spacy english and italian
# !python -m spacy download en_core_web_sm

loading english sapcy models

In [10]:
spacy = spacy.load("en_core_web_sm")

## Previewing the Dataset
Link to the dataset https://www.kaggle.com/thoughtvector/customer-support-on-twitter

In [4]:
twitter_df = pd.read_csv("./../datasets/Customer Support Datasets for Chatbot Training/customer_support_twitter/twcs/twcs.csv")
twitter_df.head(5)

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [5]:
twitter_df.shape

(2811774, 7)

Creating a list out of it, where each set has the inbound mail to company as well as it's response

In [6]:
# dictionary of inbound tweets with tweet_id
inbound_reply_tweets = {}
for idx in tqdm(twitter_df.index):
    if not twitter_df.iloc[idx]["inbound"]:
        inbound_reply_tweets[twitter_df.iloc[idx]["tweet_id"]] = twitter_df.iloc[idx]["text"]

<ipython-input-6-152a3e10c249>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm(twitter_df.index):


In [7]:
conversation_pairs = []

for idx in tqdm(twitter_df.index):
    if twitter_df.iloc[idx]["inbound"]:
        try:
            inbound_to_company_tweet = twitter_df.iloc[idx]["text"]
            
            reply_tweet = inbound_reply_tweets[int(twitter_df.iloc[idx]["response_tweet_id"])]
            conversation_pairs.append([inbound_to_company_tweet, reply_tweet])
        except:
            pass

<ipython-input-7-a3370aa58fcd>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm(twitter_df.index):


In [8]:
len(conversation_pairs)

1016105

In [71]:
conversation_pairs[10]

['My picture on @Ask_Spectrum pretty much every day. Why should I pay $171 per month? https://t.co/U6ptkQa5Ik',
 '@115718 I apologize for the inconvenience. I will be glad to assist you. Can you DM me your Name and acct # or phone #? -JB']

## Defining the Fields

Question and Answer are standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [72]:
Question = Field(sequential = True,
                      tokenize = 'spacy',
#                       batch_first =True,
                      include_lengths=True,
                      init_token = '<sos>',
                      eos_token = '<eos>',
                      lower = True)

Answer = Field(sequential = True,
                    tokenize ='spacy',
#                     batch_first =True,
                    include_lengths=True,
                    init_token = '<sos>',
                    eos_token = '<eos>',
                    lower = True)

/home/tensor/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the list quescontext_ans_pairs

In [77]:
fields = [('questions', Question),('answers',Question)]

In [78]:
example = [data.Example.fromlist([conversation_pairs[i][0],conversation_pairs[i][1]], fields) for i in range(len(conversation_pairs))] 

/home/tensor/.local/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


## Creating dataset

In [79]:
# creating dataset
twcsDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [80]:
(train, valid, test) = twcsDataset.split(split_ratio=[0.70, 0.15, 0.15], random_state=random.seed(SEED))

In [81]:
(len(train), len(valid), len(test))

(711274, 152415, 152416)

An example from the dataset:

In [82]:
vars(train.examples[10])

{'questions': ['hey',
  '@116068',
  'i',
  'like',
  'your',
  'service',
  'but',
  'your',
  'sales',
  'rep',
  'that',
  'just',
  'called',
  'me',
  'to',
  '"',
  'consult',
  '"',
  'about',
  'my',
  'sites',
  'was',
  'rude',
  ',',
  'condescending',
  ',',
  'and',
  'would',
  "n't",
  'answer',
  'my',
  'questions',
  '.'],
 'answers': ['@558851',
  'i',
  "'m",
  'sorry',
  'to',
  'hear',
  'that',
  'jessica',
  '.',
  'could',
  'you',
  'please',
  'dm',
  'us',
  'your',
  'customer',
  'number',
  'and',
  'more',
  'details',
  '?',
  'i',
  "'d",
  'like',
  'to',
  'look',
  'into',
  'this',
  '.',
  '^lisi',
  'https://t.co/ryq25s2pfj']}

## Data Augmentation
copied from the repository of Woncoh1 https://github.com/woncoh1/END/blob/main/Assignment_Session_7_Sentiment_Analysis_using_LSTM_RNN.ipynb

In [83]:
# import libraries and prepare sentence text

import random
# for back translation
import google_trans_new
from google_trans_new import google_translator
# for word tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/tensor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Back Translation

In [84]:
# translate a sentence to a random language,
# and translate back to original language

def back_translate(sentence, p=0.1):
    # do nothing with probability of 1-p
    if random.uniform(0,1) > p:
        return sentence
    
    # combine tokenized sentence into one string
    sentence = " ".join(sentence)
    
    # instantiate translator
    translator = google_translator()
    
    # choose a target language
    available_langs = list(google_trans_new.LANGUAGES.keys())
    trans_lang = random.choice(available_langs)
    
    # translate to the target language
    translations = translator.translate(sentence, lang_tgt=trans_lang)
    
    # translate back to original language
    translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt="en")
    
    # select only one translation
    if len(translations_en_random) > 1:
        translations_en_random = translations_en_random[0]
        
    return word_tokenize(translations_en_random)

#### Random Deletion

In [85]:
# randomly delete words from a sentence with a given probability

def random_deletion(sentence, p=0.5): 
    # return if single word
    if len(sentence) == 1: 
        return sentence
    # delete words
    remaining = list(filter(lambda x: random.uniform(0,1) > p, sentence)) 
    # if nothing left, sample a random word
    if len(remaining) == 0: 
        return [random.choice(sentence)] 
    else:
        return remaining

#### Random Swap


In [86]:
# randomly swap a pair of words in a sentence for a given # of times

def random_swap(sentence, n=5): 
    if len(sentence) < 2:
        return sentence
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

#### Carry Out Data Augmentation

In [88]:
for example in train.examples: 
    example.questions = back_translate(example.questions, p=0.01)
    example.questions = random_deletion(example.questions, p=0.1)
    example.questions = random_swap(example.questions, n=1)
    
    example.answers = back_translate(example.answers, p=0.01)
    example.answers = random_deletion(example.answers, p=0.01)
    example.answers = random_swap(example.answers, n=1)

## Building Vocabulary

In [89]:

MAX_VOCAB_SIZE = 25000
''' Use max of 25000 frequently occuring words to create vocabulary. Use pretrained fasttext embedding. Embedding of words that exist in the data but not in the
dictionary are set using normal distribution.
'''
Question.build_vocab(train,
                     max_size = MAX_VOCAB_SIZE,
                     vectors='fasttext.simple.300d',
                     unk_init = torch.Tensor.normal_,
                     min_freq = 2)


In [90]:
# get the vocab instance
vocab = Question.vocab

In [91]:
embedding_vector = Question.vocab.vectors
embedding_vector.shape

torch.Size([25004, 300])

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [92]:
print('Size of query vocab : ', len(Question.vocab))
# print('Size of answer vocab : ', len(Answer.vocab))
print('Top 10 words appeared in contex+question:', list(Question.vocab.freqs.most_common(10)))
# print("Top 10 words appeared in answer:", list(Answer.vocab.freqs.most_common(10)))

Size of query vocab :  25004
Top 10 words appeared in contex+question: [('.', 1372493), ('to', 839009), ('the', 707113), ('you', 700220), (',', 643614), ('i', 552799), ('we', 479400), ('?', 443443), ('a', 438426), ('!', 429986)]


## Create the iterators


In [93]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [94]:
BATCH_SIZE = 8
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train, valid, test),
                                                            batch_size = BATCH_SIZE, 
                                                            sort_key = lambda x: len(x.questions),
                                                            sort_within_batch=True,
                                                            device = device)

In [95]:
vars(Question.vocab)

{'freqs': Counter({'seràn': 1,
          'el': 6823,
          'iphone': 28010,
          'mi': 2724,
          'o': 4665,
          'a': 438426,
          'dar': 128,
          'tan': 163,
          'iphonex': 864,
          '?': 443443,
          'ya': 3264,
          'mías': 1,
          'truquito': 1,
          '@115940': 4765,
          'please': 218428,
          'add': 7405,
          'of': 168421,
          'nflredzone': 5,
          '@9833': 17,
          'to': 839009,
          'your': 366711,
          'live': 8135,
          'the': 707113,
          'this': 299304,
          'will': 86379,
          'it': 271340,
          'too': 13734,
          'notch': 214,
          '#': 120229,
          '.': 1372493,
          'guys': 14877,
          'oh': 16198,
          'and': 396071,
          'maybe': 3488,
          ' ': 113627,
          'thank': 51423,
          'you': 700220,
          '@amazonhelp': 52291,
          'just': 65830,
          "n't": 104436,
          'does': 

## Building the model

### Encoder

First, we'll build the encoder using a single layer GRU, however we now use a *bidirectional RNN*. With a bidirectional RNN, we have two RNNs in each layer. A *forward RNN* going over the embedded sentence from left to right (shown below in green), and a *backward RNN* going over the embedded sentence from right to left (teal). All we need to do in code is set `bidirectional = True` and then pass the embedded sentence to the RNN as before. 

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq8.png?raw=1)

We now have:

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(e(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(e(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

Where $x_0^\rightarrow = \text{<sos>}, x_1^\rightarrow = \text{guten}$ and $x_0^\leftarrow = \text{<eos>}, x_1^\leftarrow = \text{morgen}$.

As before, we only pass an input (`embedded`) to the RNN, which tells PyTorch to initialize both the forward and backward initial hidden states ($h_0^\rightarrow$ and $h_0^\leftarrow$, respectively) to a tensor of all zeros. We'll also get two context vectors, one from the forward RNN after it has seen the final word in the sentence, $z^\rightarrow=h_T^\rightarrow$, and one from the backward RNN after it has seen the first word in the sentence, $z^\leftarrow=h_T^\leftarrow$.

The RNN returns `outputs` and `hidden`. 

`outputs` is of size **[src len, batch size, hid dim * num directions]** where the first `hid_dim` elements in the third axis are the hidden states from the top layer forward RNN, and the last `hid_dim` elements are hidden states from the top layer backward RNN. We can think of the third axis as being the forward and backward hidden states concatenated together other, i.e. $h_1 = [h_1^\rightarrow; h_{T}^\leftarrow]$, $h_2 = [h_2^\rightarrow; h_{T-1}^\leftarrow]$ and we can denote all encoder hidden states (forward and backwards concatenated together) as $H=\{ h_1, h_2, ..., h_T\}$.

`hidden` is of size **[n layers * num directions, batch size, hid dim]**, where **[-2, :, :]** gives the top layer forward RNN hidden state after the final time-step (i.e. after it has seen the last word in the sentence) and **[-1, :, :]** gives the top layer backward RNN hidden state after the final time-step (i.e. after it has seen the first word in the sentence).

As the decoder is not bidirectional, it only needs a single context vector, $z$, to use as its initial hidden state, $s_0$, and we currently have two, a forward and a backward one ($z^\rightarrow=h_T^\rightarrow$ and $z^\leftarrow=h_T^\leftarrow$, respectively). We solve this by concatenating the two context vectors together, passing them through a linear layer, $g$, and applying the $\tanh$ activation function. 

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

**Note**: this is actually a deviation from the paper. Instead, they feed only the first backward RNN hidden state through a linear layer to get the context vector/decoder initial hidden state. This doesn't seem to make sense to me, so we have changed it.

As we want our model to look back over the whole of the source sentence we return `outputs`, the stacked forward and backward hidden states for every token in the source sentence. We also return `hidden`, which acts as our initial hidden state in the decoder.

In [96]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, emb_vector, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
#         self.embedding = nn.Embedding(input_dim, emb_dim)
        self.emb_vector = emb_vector
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
#         embedded = self.dropout(self.embedding(src))
        embedded = self.emb_vector[src]
#         embedded = embedded.to(device)
        #embedded = [src len, batch size, emb dim]
    
        embedded = embedded.to(device)
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

### Attention

Next up is the attention layer. This will take in the previous hidden state of the decoder, $s_{t-1}$, and all of the stacked forward and backward hidden states from the encoder, $H$. The layer will output an attention vector, $a_t$, that is the length of the source sentence, each element is between 0 and 1 and the entire vector sums to 1.

Intuitively, this layer takes what we have decoded so far, $s_{t-1}$, and all of what we have encoded, $H$, to produce a vector, $a_t$, that represents which words in the source sentence we should pay the most attention to in order to correctly predict the next word to decode, $\hat{y}_{t+1}$. 

First, we calculate the *energy* between the previous decoder hidden state and the encoder hidden states. As our encoder hidden states are a sequence of $T$ tensors, and our previous decoder hidden state is a single tensor, the first thing we do is `repeat` the previous decoder hidden state $T$ times. We then calculate the energy, $E_t$, between them by concatenating them together and passing them through a linear layer (`attn`) and a $\tanh$ activation function. 

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

This can be thought of as calculating how well each encoder hidden state "matches" the previous decoder hidden state.

We currently have a **[dec hid dim, src len]** tensor for each example in the batch. We want this to be **[src len]** for each example in the batch as the attention should be over the length of the source sentence. This is achieved by multiplying the `energy` by a **[1, dec hid dim]** tensor, $v$.

$$\hat{a}_t = v E_t$$

We can think of $v$ as the weights for a weighted sum of the energy across all encoder hidden states. These weights tell us how much we should attend to each token in the source sequence. The parameters of $v$ are initialized randomly, but learned with the rest of the model via backpropagation. Note how $v$ is not dependent on time, and the same $v$ is used for each time-step of the decoding. We implement $v$ as a linear layer without a bias.

Finally, we ensure the attention vector fits the constraints of having all elements between 0 and 1 and the vector summing to 1 by passing it through a $\text{softmax}$ layer.

$$a_t = \text{softmax}(\hat{a_t})$$

This gives us the attention over the source sentence!

Graphically, this looks something like below. This is for calculating the very first attention vector, where $s_{t-1} = s_0 = z$. The green/teal blocks represent the hidden states from both the forward and backward RNNs, and the attention computation is all done within the pink block.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq9.png?raw=1)

In [97]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

### Decoder

Next up is the decoder. 

The decoder contains the attention layer, `attention`, which takes the previous hidden state, $s_{t-1}$, all of the encoder hidden states, $H$, and returns the attention vector, $a_t$.

We then use this attention vector to create a weighted source vector, $w_t$, denoted by `weighted`, which is a weighted sum of the encoder hidden states, $H$, using $a_t$ as the weights.

$$w_t = a_t H$$

The embedded input word, $d(y_t)$, the weighted source vector, $w_t$, and the previous decoder hidden state, $s_{t-1}$, are then all passed into the decoder RNN, with $d(y_t)$ and $w_t$ being concatenated together.

$$s_t = \text{DecoderGRU}(d(y_t), w_t, s_{t-1})$$

We then pass $d(y_t)$, $w_t$ and $s_t$ through the linear layer, $f$, to make a prediction of the next word in the target sentence, $\hat{y}_{t+1}$. This is done by concatenating them all together.

$$\hat{y}_{t+1} = f(d(y_t), w_t, s_t)$$

The image below shows decoding the first word in an example translation.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq10.png?raw=1)

The green/teal blocks show the forward/backward encoder RNNs which output $H$, the red block shows the context vector, $z = h_T = \tanh(g(h^\rightarrow_T,h^\leftarrow_T)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$, the blue block shows the decoder RNN which outputs $s_t$, the purple block shows the linear layer, $f$, which outputs $\hat{y}_{t+1}$ and the orange block shows the calculation of the weighted sum over $H$ by $a_t$ and outputs $w_t$. Not shown is the calculation of $a_t$.

In [98]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, emb_vector, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
#         self.embedding = nn.Embedding(output_dim, emb_dim)
        self.emb_vector = emb_vector
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
#         embedded = self.dropout(self.embedding(input))
        embedded = self.emb_vector[input]
        embedded = embedded.to(device)
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

### Seq2Seq

In this model we don't have to have the encoder RNN and decoder RNN have the same hidden dimensions, however the encoder has to be bidirectional. This requirement can be removed by changing all occurences of `enc_dim * 2` to `enc_dim * 2 if encoder_is_bidirectional else enc_dim`. 

This seq2seq encapsulator is that the `encoder` returns both the final hidden state (which is the final hidden state from both the forward and backward encoder RNNs passed through a linear layer) to be used as the initial hidden state for the decoder, as well as every hidden state (which are the forward and backward hidden states stacked on top of each other). We also need to ensure that `hidden` and `encoder_outputs` are passed to the decoder. 

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive $z$ and $H$
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and all encoder outputs, $H$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate

In [99]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

We initialise our parameters, encoder, decoder and seq2seq model (placing it on the GPU if we have one). 

In [100]:
len(Question.vocab)

25004

In [101]:
INPUT_DIM = len(Question.vocab)
OUTPUT_DIM = len(Question.vocab)
# ENC_EMB_DIM = 256
ENC_EMB_DIM = 300
# DEC_EMB_DIM = 256
DEC_EMB_DIM = 300
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, embedding_vector, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, embedding_vector, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [102]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (rnn): GRU(300, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (rnn): GRU(1324, 512)
    (fc_out): Linear(in_features=1836, out_features=25004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Calculate the number of parameters

In [103]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 52,568,380 trainable parameters


We create an optimizer.

In [104]:
optimizer = optim.Adam(model.parameters())

We initialize the loss function.

In [105]:
ANSWER_PAD_IDX = Question.vocab.stoi[Question.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = ANSWER_PAD_IDX)

We then create the training loop...

In [106]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src, src_length = batch.questions
        trg, trg_length = batch.answers
        

        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        if(i % 1000 == 0):
            print(f"{i} steps are done")
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [107]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, src_length = batch.questions
            trg, trg_length = batch.answers

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, define a timing function.

In [108]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best 


In [109]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'squad-attention.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

/home/tensor/.local/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


0 steps are done
1000 steps are done
2000 steps are done
3000 steps are done
4000 steps are done
5000 steps are done
6000 steps are done
7000 steps are done
8000 steps are done
9000 steps are done
10000 steps are done
11000 steps are done
12000 steps are done
13000 steps are done
14000 steps are done
15000 steps are done
16000 steps are done
17000 steps are done
18000 steps are done
19000 steps are done
20000 steps are done
21000 steps are done
22000 steps are done
23000 steps are done
24000 steps are done
25000 steps are done
26000 steps are done
27000 steps are done
28000 steps are done
29000 steps are done
30000 steps are done
31000 steps are done
32000 steps are done
33000 steps are done
34000 steps are done
35000 steps are done
36000 steps are done
37000 steps are done
38000 steps are done
39000 steps are done
40000 steps are done
41000 steps are done
42000 steps are done
43000 steps are done
44000 steps are done
45000 steps are done
46000 steps are done
47000 steps are done
48000

KeyboardInterrupt: 

Finally, we test the model on the test set using these "best" parameters.

In [110]:
model.load_state_dict(torch.load('squad-attention.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.736 | Test PPL: 309.920 |
